In [154]:
import torch
import random
import time
import numpy as np
import pandas as pd
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pickle
import glob
import math
import pandas as pd
import collections
import random
from torch_geometric.nn import GCNConv
from torch_geometric.data import Batch

## Read demand data

In [155]:
def data_reader(filename):
    f = open(filename,'rb')
    data = pickle.load(f)
    return data

In [156]:
filename = glob.glob('../data/demand/*')
all_data = {}
node_feature = []
for i in filename:
    num = i.split('_')[1]
    data = data_reader(i)
    all_data[int(num)] = data
    node_feature.append(data)
sorted_data = collections.OrderedDict(sorted(all_data.items()))
node_feature = np.array(node_feature)

In [157]:
node_feature.shape

(262, 35064)

In [158]:
all_key = list(sorted_data.keys())

In [159]:
oldkey2newkey = {all_key[i] : i for i in range(len(all_key))}

## Read graph structure¶

In [160]:
file = open('../data/proxList','rb')
data = pickle.load(file)

In [161]:
edge_index = [[], []]
for item in data:
    edge_index[0].append(item[0])
    edge_index[1].append(item[1])

In [162]:
initial_feature = node_feature.mean()

In [163]:
node_feature.mean(1).shape

(262,)

## Clean graph

In [164]:
new_graph = [[], []]
for i in range(len(edge_index[0])):
    if edge_index[0][i] in all_key and edge_index[1][i] in all_key:
        new_graph[0].append(oldkey2newkey[edge_index[0][i]])
        new_graph[1].append(oldkey2newkey[edge_index[1][i]])

In [165]:
edge_index = torch.tensor(new_graph, dtype=torch.long)
node_feature = torch.tensor(node_feature, dtype=torch.float32)

## Hyperparameters

In [279]:
class Args():
    def __init__(self):
        self.split_ratio = [0.7, 0.2, 0.1]
        self.hist_length = 24 
        self.pred_length = 12
        self.batch_size = 1
        self.num_node_features = 1
        self.gcn_hidden_dim = 16
        self.encoder_hidden_dim = 16
        self.decoder_hidden_dim = 16
        self.zone_num = 1
        # self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.device = torch.device("cpu")
        self.epoch_num = 30

## Dataset

In [280]:
class GraphDataset(Dataset):
    def __init__(self, node_feature, hist_length, pred_length):
        self.node_feature = node_feature
        self.total_len = hist_length + pred_length
        self.hist_len = hist_length
        self.pred_len = pred_length
        
    def __len__(self):
        return int(self.node_feature.shape[1] - self.total_len)
    
    def __getitem__(self, index):
        return (self.node_feature[:, index:index + self.hist_len], 
            self.node_feature[:, index + self.hist_len: index + self.total_len])

In [281]:
total_len = node_feature.shape[1]
train_st = 0
train_ed = math.floor(total_len * args.split_ratio[0])
valid_st = math.floor(total_len * args.split_ratio[0])
valid_ed = math.floor(total_len * (args.split_ratio[0] + args.split_ratio[1]))
test_st = math.floor(total_len * (args.split_ratio[0] + args.split_ratio[1]))
test_ed = total_len
train_dataset = GraphDataset(node_feature[:, train_st:train_ed], args.hist_length, args.pred_length)
val_dataset = GraphDataset(node_feature[:, valid_st:valid_ed], args.hist_length, args.pred_length)
test_dataset = GraphDataset(node_feature[:, test_st:test_ed], args.hist_length, args.pred_length)

In [282]:
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [283]:
len(train_dataset)

24508

In [284]:
train_dataset[24510][0].shape

torch.Size([262, 24])

## Model

In [285]:
class GCN(torch.nn.Module):
    def __init__(self, args):
        super().__init__()
        self.conv1 = GCNConv(args.num_node_features, args.gcn_hidden_dim)
        self.conv2 = GCNConv(args.gcn_hidden_dim, args.gcn_hidden_dim)

    def forward(self, node_feature, edge_index):
        x = self.conv1(node_feature, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [286]:
class GRU_Encoder(torch.nn.Module):
    def __init__(self, args):
        super().__init__()
        self.feature_extractor = GCN(args)
        self.gru1 = nn.GRU(input_size=args.gcn_hidden_dim, hidden_size=args.encoder_hidden_dim)
        self.gru2 = nn.GRU(input_size=args.encoder_hidden_dim, hidden_size=args.encoder_hidden_dim)
        
    def forward(self, batchx, edge_index):
        GCN_output = []
        for i in range(batchx[0].shape[1]):
            gcn_feature = self.feature_extractor(torch.unsqueeze(batchx[0][:, i], 1), edge_index)
            GCN_output.append(gcn_feature)
        
        out = torch.stack(GCN_output)
        out, h1 = self.gru1(out)
        out, h2 = self.gru2(out)
        return h1, h2

In [287]:
class GRU_Decoder(torch.nn.Module):
    def __init__(self, args):
        super().__init__()
        self.gcn = GCN(args)
        self.gru1 = nn.GRUCell(input_size=args.gcn_hidden_dim, hidden_size=args.decoder_hidden_dim)
        self.gru2 = nn.GRUCell(input_size=args.decoder_hidden_dim, hidden_size=args.decoder_hidden_dim)
        self.pred = nn.Linear(args.decoder_hidden_dim, args.zone_num)
    
    def forward(self, input_graph, edge_index, h1, h2):
        graph_embed = self.gcn(input_graph, edge_index)
        h1 = self.gru1(graph_embed, h1)
        h2 = self.gru2(torch.squeeze(h1), h2)
        out_graph = self.pred(torch.squeeze(h2))
        return (out_graph, h1, h2)

In [288]:
class Graph_Seq2Seq(torch.nn.Module):
    def __init__(self, args, initial_feature):
        super().__init__()
        self.initial_feature = initial_feature
        self.encoder = GRU_Encoder(args)
        self.decoder = GRU_Decoder(args)
        
    def forward(self, input_seq, edge_index, pred_len):
        graph_li = []
        h1, h2 = self.encoder(input_seq, edge_index)
        graph = self.initial_feature
        for i in range(pred_len):
            out_graph, h1, h2 = self.decoder(graph, edge_index, torch.squeeze(h1), torch.squeeze(h2))
            graph_li.append(out_graph)
        pred = torch.stack(graph_li)
        return pred

In [289]:
#initial_feature = torch.unsqueeze(node_feature.mean(1), 1)
#model = Graph_Seq2Seq(args, initial_feature)
#for (batchx, batchy) in train_loader:
#    x = model(batchx, edge_index, 12)
#    print(x)
#    i += 1

In [290]:
args.device

device(type='cpu')

In [294]:
def train(model, criterion, optimizer, edge_index, args):
    model.train()
    eval_loss = []
    edge_index = edge_index.to(args.device)
    for batchIdx, (batchx, batchy) in enumerate(train_loader):
        optimizer.zero_grad()
        batchx = batchx.to(args.device)
        batchy = batchy.to(args.device)
        predictions = model(batchx, edge_index, args.pred_length)
        loss = criterion(predictions, batchy.permute(2, 1, 0))
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
        optimizer.step()
        eval_loss.append(loss.detach().numpy())
        end = time.time()
        print('train loss:', loss)
    return np.mean(eval_loss)

In [295]:
def valid(model, criterion, edge_index, args):
    model.eval()
    eval_loss = []
    edge_index = edge_index.to(args.device)
    for batchIdx, (batchx, batchy) in enumerate(train_loader):
        optimizer.zero_grad()
        batchx = batchx.to(args.device)
        batchy = batchy.to(args.device)
        predictions = model(batchx, edge_index, args.pred_length)
        loss = criterion(predictions, batchy.permute(2, 1, 0))
        eval_loss.append(loss.detach().numpy())
        end = time.time()
    return np.mean(eval_loss)

In [ ]:
model = Graph_Seq2Seq(args, initial_feature).to(args.device)
criterion = F.mse_loss
optimizer = optim.Adam(model.parameters(), amsgrad=True)

for i in range(args.epoch_num):
    train_loss = train(model, criterion, optimizer, edge_index, args)
    print('----------------------------- epoch {} train loss {} ------------------------------'.format(i, train_loss))
    eval_loss = valid(model, criterion, edge_index, args)
    print('----------------------------- epoch {} eval loss {} ------------------------------'.format(i, eval_loss))

train loss: tensor(13674.5430, grad_fn=<MseLossBackward0>)
train loss: tensor(13369.1904, grad_fn=<MseLossBackward0>)
train loss: tensor(15390.6553, grad_fn=<MseLossBackward0>)
train loss: tensor(24052.2910, grad_fn=<MseLossBackward0>)
train loss: tensor(18861.7793, grad_fn=<MseLossBackward0>)
train loss: tensor(4009.0461, grad_fn=<MseLossBackward0>)
train loss: tensor(8033.1475, grad_fn=<MseLossBackward0>)
train loss: tensor(29382.9355, grad_fn=<MseLossBackward0>)
train loss: tensor(9519.3779, grad_fn=<MseLossBackward0>)
train loss: tensor(9859.7383, grad_fn=<MseLossBackward0>)
train loss: tensor(21745.8828, grad_fn=<MseLossBackward0>)
train loss: tensor(32618.3633, grad_fn=<MseLossBackward0>)
train loss: tensor(21214.6113, grad_fn=<MseLossBackward0>)
train loss: tensor(15384.6182, grad_fn=<MseLossBackward0>)
train loss: tensor(14029.1504, grad_fn=<MseLossBackward0>)
train loss: tensor(7277.2632, grad_fn=<MseLossBackward0>)
train loss: tensor(11530.6338, grad_fn=<MseLossBackward0>)
tr

train loss: tensor(15649.9414, grad_fn=<MseLossBackward0>)
train loss: tensor(9899.0996, grad_fn=<MseLossBackward0>)
train loss: tensor(11736.7773, grad_fn=<MseLossBackward0>)
train loss: tensor(21200.2754, grad_fn=<MseLossBackward0>)
train loss: tensor(6441.0562, grad_fn=<MseLossBackward0>)
train loss: tensor(13285.6768, grad_fn=<MseLossBackward0>)
train loss: tensor(12105.6230, grad_fn=<MseLossBackward0>)
train loss: tensor(30710.2383, grad_fn=<MseLossBackward0>)
train loss: tensor(14461.2852, grad_fn=<MseLossBackward0>)
train loss: tensor(18063.6230, grad_fn=<MseLossBackward0>)
train loss: tensor(26250.0781, grad_fn=<MseLossBackward0>)
train loss: tensor(7253.3721, grad_fn=<MseLossBackward0>)
train loss: tensor(11006.0088, grad_fn=<MseLossBackward0>)
train loss: tensor(7810.1851, grad_fn=<MseLossBackward0>)
train loss: tensor(9176.2734, grad_fn=<MseLossBackward0>)
train loss: tensor(14970.5156, grad_fn=<MseLossBackward0>)
train loss: tensor(26498.7988, grad_fn=<MseLossBackward0>)
tr

train loss: tensor(10136.6143, grad_fn=<MseLossBackward0>)
train loss: tensor(12623.7695, grad_fn=<MseLossBackward0>)
train loss: tensor(13178.4922, grad_fn=<MseLossBackward0>)
train loss: tensor(13673.7383, grad_fn=<MseLossBackward0>)
train loss: tensor(34211.0898, grad_fn=<MseLossBackward0>)
train loss: tensor(11794.6348, grad_fn=<MseLossBackward0>)
train loss: tensor(29963.9805, grad_fn=<MseLossBackward0>)
train loss: tensor(21524.3262, grad_fn=<MseLossBackward0>)
train loss: tensor(29249.4141, grad_fn=<MseLossBackward0>)
train loss: tensor(11504.3691, grad_fn=<MseLossBackward0>)
train loss: tensor(6943.4517, grad_fn=<MseLossBackward0>)
train loss: tensor(18603.4297, grad_fn=<MseLossBackward0>)
train loss: tensor(9534.0693, grad_fn=<MseLossBackward0>)
train loss: tensor(8240.9805, grad_fn=<MseLossBackward0>)
train loss: tensor(11216.0127, grad_fn=<MseLossBackward0>)
train loss: tensor(12501.8877, grad_fn=<MseLossBackward0>)
train loss: tensor(9532.6484, grad_fn=<MseLossBackward0>)
t

train loss: tensor(20038.7969, grad_fn=<MseLossBackward0>)
train loss: tensor(19260.3477, grad_fn=<MseLossBackward0>)
train loss: tensor(9622.4717, grad_fn=<MseLossBackward0>)
train loss: tensor(27476.7227, grad_fn=<MseLossBackward0>)
train loss: tensor(9506.5967, grad_fn=<MseLossBackward0>)
train loss: tensor(10849.9365, grad_fn=<MseLossBackward0>)
train loss: tensor(14942.6611, grad_fn=<MseLossBackward0>)
train loss: tensor(9016.5225, grad_fn=<MseLossBackward0>)
train loss: tensor(18647.5645, grad_fn=<MseLossBackward0>)
train loss: tensor(10854.4629, grad_fn=<MseLossBackward0>)
train loss: tensor(17451.7891, grad_fn=<MseLossBackward0>)
train loss: tensor(9253.2305, grad_fn=<MseLossBackward0>)
train loss: tensor(10960.0752, grad_fn=<MseLossBackward0>)
train loss: tensor(25792.0664, grad_fn=<MseLossBackward0>)
train loss: tensor(39678.6797, grad_fn=<MseLossBackward0>)
train loss: tensor(7124.7900, grad_fn=<MseLossBackward0>)
train loss: tensor(10025.5537, grad_fn=<MseLossBackward0>)
tr

train loss: tensor(19785.9844, grad_fn=<MseLossBackward0>)
train loss: tensor(20295.9727, grad_fn=<MseLossBackward0>)
train loss: tensor(11260.7363, grad_fn=<MseLossBackward0>)
train loss: tensor(4353.3838, grad_fn=<MseLossBackward0>)
train loss: tensor(26254.3262, grad_fn=<MseLossBackward0>)
train loss: tensor(29785.0352, grad_fn=<MseLossBackward0>)
train loss: tensor(11142.2314, grad_fn=<MseLossBackward0>)
train loss: tensor(19795.4707, grad_fn=<MseLossBackward0>)
train loss: tensor(8172.7661, grad_fn=<MseLossBackward0>)
train loss: tensor(8922.6543, grad_fn=<MseLossBackward0>)
train loss: tensor(7964.5864, grad_fn=<MseLossBackward0>)
train loss: tensor(7558.7925, grad_fn=<MseLossBackward0>)
train loss: tensor(12297.1582, grad_fn=<MseLossBackward0>)
train loss: tensor(24394.9922, grad_fn=<MseLossBackward0>)
train loss: tensor(22630.2715, grad_fn=<MseLossBackward0>)
train loss: tensor(19485.2891, grad_fn=<MseLossBackward0>)
train loss: tensor(14106.6582, grad_fn=<MseLossBackward0>)
tr

...

train loss: tensor(4792.0542, grad_fn=<MseLossBackward0>)
train loss: tensor(6157.7207, grad_fn=<MseLossBackward0>)
train loss: tensor(4275.8906, grad_fn=<MseLossBackward0>)
train loss: tensor(6218.6982, grad_fn=<MseLossBackward0>)
train loss: tensor(3012.6621, grad_fn=<MseLossBackward0>)
train loss: tensor(4216.1411, grad_fn=<MseLossBackward0>)
train loss: tensor(5709.6094, grad_fn=<MseLossBackward0>)
train loss: tensor(1363.2976, grad_fn=<MseLossBackward0>)
train loss: tensor(10401.4092, grad_fn=<MseLossBackward0>)
train loss: tensor(4802.7334, grad_fn=<MseLossBackward0>)
train loss: tensor(9393.0586, grad_fn=<MseLossBackward0>)
train loss: tensor(4020.0349, grad_fn=<MseLossBackward0>)
train loss: tensor(3917.6453, grad_fn=<MseLossBackward0>)
train loss: tensor(6675.4575, grad_fn=<MseLossBackward0>)
train loss: tensor(15172.1729, grad_fn=<MseLossBackward0>)
train loss: tensor(2755.9329, grad_fn=<MseLossBackward0>)
train loss: tensor(3422.3843, grad_fn=<MseLossBackward0>)
train loss: 

train loss: tensor(8065.5864, grad_fn=<MseLossBackward0>)
train loss: tensor(9550.8682, grad_fn=<MseLossBackward0>)
train loss: tensor(6542.6538, grad_fn=<MseLossBackward0>)
train loss: tensor(38517.7891, grad_fn=<MseLossBackward0>)
train loss: tensor(5510.4731, grad_fn=<MseLossBackward0>)
train loss: tensor(10910.8701, grad_fn=<MseLossBackward0>)
train loss: tensor(6930.9302, grad_fn=<MseLossBackward0>)
train loss: tensor(8926.5410, grad_fn=<MseLossBackward0>)
train loss: tensor(3079.1672, grad_fn=<MseLossBackward0>)
train loss: tensor(2736.8594, grad_fn=<MseLossBackward0>)
train loss: tensor(3541.0337, grad_fn=<MseLossBackward0>)
train loss: tensor(7407.6895, grad_fn=<MseLossBackward0>)
train loss: tensor(9513.8418, grad_fn=<MseLossBackward0>)
train loss: tensor(8466.5088, grad_fn=<MseLossBackward0>)
train loss: tensor(5272.9985, grad_fn=<MseLossBackward0>)
train loss: tensor(12403.5928, grad_fn=<MseLossBackward0>)
train loss: tensor(4546.8926, grad_fn=<MseLossBackward0>)
train loss:

train loss: tensor(3232.4460, grad_fn=<MseLossBackward0>)
train loss: tensor(13341.4502, grad_fn=<MseLossBackward0>)
train loss: tensor(12541.5977, grad_fn=<MseLossBackward0>)
train loss: tensor(6753.8066, grad_fn=<MseLossBackward0>)
train loss: tensor(5704.5586, grad_fn=<MseLossBackward0>)
train loss: tensor(2553.5305, grad_fn=<MseLossBackward0>)
train loss: tensor(5253.9087, grad_fn=<MseLossBackward0>)
train loss: tensor(4834.4175, grad_fn=<MseLossBackward0>)
train loss: tensor(6891.3779, grad_fn=<MseLossBackward0>)
train loss: tensor(4809.1411, grad_fn=<MseLossBackward0>)
train loss: tensor(5713.0132, grad_fn=<MseLossBackward0>)
train loss: tensor(2684.3633, grad_fn=<MseLossBackward0>)
train loss: tensor(4144.4731, grad_fn=<MseLossBackward0>)
train loss: tensor(8263.9883, grad_fn=<MseLossBackward0>)
train loss: tensor(9282.8730, grad_fn=<MseLossBackward0>)
train loss: tensor(2877.8047, grad_fn=<MseLossBackward0>)
train loss: tensor(5570.9531, grad_fn=<MseLossBackward0>)
train loss: 

train loss: tensor(8782.0527, grad_fn=<MseLossBackward0>)
train loss: tensor(9515.5029, grad_fn=<MseLossBackward0>)
train loss: tensor(5968.9019, grad_fn=<MseLossBackward0>)
train loss: tensor(6471.1138, grad_fn=<MseLossBackward0>)
train loss: tensor(4079.5745, grad_fn=<MseLossBackward0>)
train loss: tensor(3813.7871, grad_fn=<MseLossBackward0>)
train loss: tensor(4413.0903, grad_fn=<MseLossBackward0>)
train loss: tensor(5747.3252, grad_fn=<MseLossBackward0>)
train loss: tensor(6962.7231, grad_fn=<MseLossBackward0>)
train loss: tensor(6341.3701, grad_fn=<MseLossBackward0>)
train loss: tensor(7284.6870, grad_fn=<MseLossBackward0>)
train loss: tensor(8027.4873, grad_fn=<MseLossBackward0>)
train loss: tensor(6730.9829, grad_fn=<MseLossBackward0>)
train loss: tensor(45594.9766, grad_fn=<MseLossBackward0>)
train loss: tensor(13518.3682, grad_fn=<MseLossBackward0>)
train loss: tensor(2653.4778, grad_fn=<MseLossBackward0>)
train loss: tensor(6802.2295, grad_fn=<MseLossBackward0>)
train loss: 

train loss: tensor(4526.3418, grad_fn=<MseLossBackward0>)
train loss: tensor(8633.1807, grad_fn=<MseLossBackward0>)
train loss: tensor(5611.3096, grad_fn=<MseLossBackward0>)
train loss: tensor(4709.4512, grad_fn=<MseLossBackward0>)
train loss: tensor(5498.8545, grad_fn=<MseLossBackward0>)
train loss: tensor(6281.4111, grad_fn=<MseLossBackward0>)
train loss: tensor(87302.0391, grad_fn=<MseLossBackward0>)
train loss: tensor(6482.1343, grad_fn=<MseLossBackward0>)
train loss: tensor(3481.8145, grad_fn=<MseLossBackward0>)
train loss: tensor(6733.2168, grad_fn=<MseLossBackward0>)
train loss: tensor(5511.4038, grad_fn=<MseLossBackward0>)
train loss: tensor(14392.2422, grad_fn=<MseLossBackward0>)
train loss: tensor(4440.6611, grad_fn=<MseLossBackward0>)
train loss: tensor(7058.2969, grad_fn=<MseLossBackward0>)
train loss: tensor(1688.1133, grad_fn=<MseLossBackward0>)
train loss: tensor(4733.5923, grad_fn=<MseLossBackward0>)
train loss: tensor(9057.3730, grad_fn=<MseLossBackward0>)
train loss: 